In [ ]:
# Cell 1 - config & imports (edit paths)
import os, glob, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ---------- EDIT THESE ----------
SHOT_NAME = "ChipShot"   # human name for logs
SOURCE_DIR = "data/preprocessed_ml/"   # folder with the per-trial preprocessed CSVs (envelope/filtered outputs)
FEATURE_DIR = "data/features_per_shot_chip_v2" # folder where features_master.csv lives
FEATURE_CSV = os.path.join(FEATURE_DIR, "features_master.csv")
GMM_LABELED = os.path.join(FEATURE_DIR, "features_master_labeled_with_tentative.csv")
OUT_DIR = os.path.join(FEATURE_DIR, "verification_report")
os.makedirs(OUT_DIR, exist_ok=True)

print("SHOT_NAME:", SHOT_NAME)
print("SOURCE_DIR:", SOURCE_DIR)
print("FEATURE_CSV:", FEATURE_CSV)
print("GMM_LABELED:", GMM_LABELED)
print("OUT_DIR:", OUT_DIR)
